# Analyzing Global Sea Level in GFDL Ocean Models

<i>John Krasting, Jacob Steinberg, on behalf of the GFDL Oceans and Cryosphere Division</i>

In this notebook, we will conduct a simple analysis of global sea level to demonstrate the capabilities of GFDL's models and the domain knowledge needed to interpret the modeling products.

### Contemporary Sea Level Budget

Observed global sea level is rise is dominated by two main factors:

<img src="https://www.ipcc.ch/report/ar6/wg1/downloads/figures/IPCC_AR6_WGI_CCBox_9_1_Figure_1.png">

* Ocean warming and thermal expansion
* Ice Sheet melt contributions

### Processes Driving Sea Level Change

In addition to these two primary drivers, other factors influence sea level at the regional and local levels.

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*UYQ1hzn-1cBp_3NYlQir6A.png">

Some of these features are represented in GFDL's ocean and climate models:
* Ocean warming
* Ocean circulation changes
* Hydrological changes (precipatation, evaporation, runoff)
* Atmospheric phenomenon (winds, storms)

Others are areas of active development
* Coupled interactive ice sheet models
* Land processes (salt intrusion, innundation, groundwater extraction)

<i>To date, no global climate model represents all of the drivers of sea level change.</i>

### Accessing GFDL's Climate Model Output

GFDL participates in the Coupled Model Intercomparison Project (CMIP). This is one of the more visibile products GFDL contributes to scientific communitity, enabling fundamental and applied research of climate change and impacts as well as informing major synthesis reports, such as the IPCC and National Climate Assessment.

GFDL had versions of the OM4 ocean model that participated in CMIP6:
* GFDL-CM4 (0.25-degree eddy-permitting horizontal resolution)
* GFDL-ESM4 (0.5-degree horizontal resolution)
* GFDL-SPEAR (1.0-degree horizontal resolution)

<i>Here, we will use results from the ESM4 model which provides a compromise between resolution and data volume</i>

We will also access GFDL's data through <a href="https://console.cloud.google.com/marketplace/product/noaa-public/cmip6">NOAA's Public Datasets on Google Cloud</a>  CMIP6 is organized through the World Climate Research Programme (WCRP) and the data on Google Cloud were processed and uploaded by the Pangeo Project.  (Similar data also exist on Amazon S3)

<img src="https://i.ytimg.com/vi/10rzdGCAIYY/mqdefault.jpg">

### Software Tools

This Jupyter notebook will use a number of Python-based analysis tools:
* Standard libraries (NumPy, Pandas, Matplotlib)
* xarray
* momlevel

The `momlevel` package is a collection of sea-level specific routines developed at GFDL to aid the analysis of MOM6 ocean model results <i>(Krasting et al., in revision)</i>  For more details, see https://momlevel.readthedocs.io/en/v0.0.7/objective.html

### Setup the Software Environment

In [ ]:
# Do this once!  After installing these tools, go to:
#    Runtime --> Restart Session

! pip  install momlevel intake intake-esm

In [ ]:
# Load software packages
import xarray as xr

xr.set_options(display_style="html")

import fsspec
import gcsfs
import intake
import matplotlib.pyplot as plt
import momlevel
import nc_time_axis
import numpy as np
import pandas as pd

%matplotlib inline

### Load GFDL-ESM4 Sea-Surface Height (`zos`) Data from Google Cloud

In [ ]:
# Load the catalog of datasets on Google Cloud
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col

In [ ]:
# Refine data to our requested list
datasets = col.search(
    experiment_id=["historical"],
    table_id=["Ofx", "Omon"],
    variable_id=["areacello", "zos"],
    grid_label="gn",
    member_id="r1i1p1f1",
    source_id="GFDL-ESM4",
)

df = datasets.df
df

In [ ]:
# Connect to Google Cloud
fs = gcsfs.GCSFileSystem(token="anon", access="read_only")

# Open Zarr stores
zstores = df.zstore.values
mappers = [fs.get_mapper(x) for x in zstores]

# Convert to Xarray Dataset
ds = [xr.open_zarr(x, consolidated=True) for x in mappers]
ds = xr.merge(ds)

# Define a wet mask for use later
ds["wet"] = xr.where(ds.zos.isel(time=0).isnull(), 0.0, 1.0)

ds

In [ ]:
# Make a simple plot to verify areacello and zos look reasonable
fig = plt.figure(figsize=(8, 2), dpi=150)

ax1 = plt.subplot(1, 2, 1)
ds.areacello.plot(ax=ax1)
_ = ax1.set_title("Cell Area")

ax2 = plt.subplot(1, 2, 2, facecolor="gray")
ds.zos.isel(time=-1).plot(ax=ax2)
_ = ax2.set_title("Dynamic Sea Surface Height")

plt.subplots_adjust(wspace=0.5)

### Extract Data at US Tide Gauge Locations

In consultation with CO-OPS/NOS, the `momlevel` package includes routines for extracting time series of sea level for many "high priority" US Tide Gauge locations:

<img src="https://momlevel.readthedocs.io/en/v0.0.7/_images/us_tide_gauge_locations.png">

In [ ]:
ds_tide_gauge = momlevel.extract_tidegauge(
    ds.zos,
    ds.lon,
    ds.lat,
    mask=ds.wet,
    threshold=75.0
)

ds_tide_gauge

In [ ]:
location = "NEW_YORK"

# Fetch the data
data = ds_tide_gauge[location].sel(time=slice("1981-01-01","2014-12-31")).load()
data = data - data[0]
data_annual = momlevel.util.annual_average(data)

# Make a plot
fig = plt.figure(figsize=(6,2.5), dpi=150)
ax = plt.subplot(1,1,1)
ax.plot(data.time,data,linewidth=0.4,color="gray")
ax.plot(data_annual.time,data_annual,linewidth=0.8,color="red", label=location)
ax.set_title("GFDL-ESM4 Dynamic Sea Surface Height Anomaly")
ax.set_ylabel("m")
ax.legend(loc=2)

### Global Dynamic Sea Surface Height (`zos`)

In [ ]:
ssh_global_monthly = ds.zos.weighted(ds.areacello).mean(("y","x")).load()
ssh_global_annual = momlevel.util.annual_average(ssh_global_monthly).load()

In [ ]:
fig = plt.figure(figsize=(8,4), dpi=150)
ax = plt.subplot(1,1,1)

ax.plot(ssh_global_monthly.time.values,ssh_global_monthly.values, linewidth=0.5, color="lightgray")
ax.plot(ssh_global_annual.time.values,ssh_global_annual.values, linewidth=1.2, color="red")

ax.set_title("Global Mean Dynamic Sea Surface Height")
ax.set_ylim(-1,1)
ax.set_ylabel("m")

<div style="background-color:pink; padding: 10px; width: 90%; border-radius: 15px;">

#### <i>Key Takeaways So Far:</i>
* The sea surface height field defined by CMIP has a very special definition: it is relative to a global mean of zero.
* Not knowing this, plotting `zos` gives you a <i>relative</i> change in the <i>dynamic</i> and <i>freshwater</i> contributions to sea level.
* Further, this is a Boussinesq (volume-conserving) model. The thermosteric component of SLR is not included in this variable and must be diagnosed offline
</div>


### Plot the Global Steric Sea Level Change

In order to the magnitude of the global steric change, we must consider the warming of the ocean. We will calculate a density change due to temperature and salinity changes in the model and convert that to an effective column height change using the model's reference density ($p_0$ = 1035 kg m-3).



From: Griffies and Greatbatch 2012.

We now need the following fields:
* 4D potential temperature
* 4D salinity
* 4D ocean model cell volume

In [ ]:
datasets = col.search(
    experiment_id=["historical"],
    table_id=["Ofx", "Omon"],
    variable_id=["volcello", "thetao", "so", "areacello"],
    grid_label="gn",
    member_id="r1i1p1f1",
    source_id="GFDL-ESM4",
)

df = datasets.df
df

In [ ]:
# Connect to Google Cloud
fs = gcsfs.GCSFileSystem(token='anon', access='read_only')

# Open Zarr stores
zstores = df.zstore.values
mappers = [fs.get_mapper(x) for x in zstores]

# Convert to Xarray Dataset
ds = [xr.open_zarr(x, consolidated=True) for x in mappers]
ds = xr.merge(ds)

# We also need the model's bathymetery for partial bottom cells
ds["deptho"] = xr.open_zarr("https://extranet.gfdl.noaa.gov/~John.Krasting/models/GFDL-ESM4/deptho").deptho.load()
ds

In [ ]:
# Use momlevel to calculate the steric change over the last 30 years of the simulation

steric = momlevel.steric(
    ds.isel(time=slice(-360, None)), domain="global", coord_names={"z": "lev"}
)

steric_change = steric[0].steric.load()

In [ ]:
# Make a plot
fig = plt.figure(figsize=(6,4))
ax = plt.subplot(1,1,1)
ax.plot(steric_change.time, steric_change*100)
ax.grid(True)
ax.set_title("GFDL-ESM4 Global Steric Sea Level Rise")
ax.set_ylabel("cm")

### Comparison With Satellite Observations

<i>From NOAA/NESDIS:</i><br>
<img src="https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/slr/slr_sla_gbl_free_txj1j2_90_500.png">

### Building Our Next Generation Model - OM5

* Sea level science is a driving application of the model development
* Non-Boussinesq (R. Hallberg, J. Steinberg)
* Dynamic Ice Sheets (O. Sergienko, A. Huth, N. Schlegel, M. Harrison, and R. Hallberg)

<a href="https://drive.google.com/drive/folders/1o2E_Z2bIcBwvuZMts34JVEGqyE_QD_Zx">.</a>